In [375]:
reload(datasets)
reload(cell_representations)
reload(samplers)

<module 'Heimdall.samplers' from '/work/magroup/shahula/single_cell_foundation_models/Heimdall/Heimdall/samplers.py'>

In [376]:
from importlib import reload
from Heimdall import cell_representations
from Heimdall import samplers
from Heimdall import datasets

In [377]:
from Heimdall.cell_representations import CellRepresentation, PartitionedCellRepresentation
from Heimdall.datasets import PartitionedDataset, PretrainDataset
from Heimdall.samplers import PartitionedDistributedSampler

import hydra
import torch
from torch.utils.data import DataLoader
from Heimdall.utils import heimdall_collate_fn
from pathlib import Path
import numpy as np

In [378]:
with hydra.initialize(version_base=None, config_path="../config"):                                                                                                                                                                                                                             
    config = hydra.compose(                                                                    
        config_name="config",                                                                  
        overrides=[                                                                            
            # "+experiments_dev=classification_experiment_dev",                                
            "+experiments=pretraining",                                              
            "user=lane-shared",                                                             
            # "model=transformer",                                                               
            "fg=identity",                                                                     
            # "fe=identity",                                                                     
            # "fc=uce",                                                                          
            # "seed=55",                                                                         
            # "project_name=demo",                                                               
            # "tasks.args.epochs=1",                                                             
            # "fc.args.max_input_length=512",                                                    
            # "fc.args.tailor_config.args.sample_size=450",                                      
            # f"user={os.environ['HYDRA_USER']}"                                               
        ],                                                                                     
    )                      

In [379]:
cr = PartitionedCellRepresentation(config)

> Found already preprocessed anndata: /scratch/heimdall/shared/cache/new_sctab_full/preprocessed_anndata/6a4725f5-f683-4a88-21f5-c1b54981659b/data.h5ad
> Finished Processing Anndata Object:
AnnData object with n_obs × n_vars = 10000 × 19331 backed at '/scratch/heimdall/shared/cache/new_sctab_full/preprocessed_anndata/6a4725f5-f683-4a88-21f5-c1b54981659b/data.h5ad'
    obs: 'soma_joinid', 'is_primary_data', 'dataset_id', 'donor_id', 'assay', 'cell_type', 'development_stage', 'disease', 'tissue', 'tissue_general', 'split', 'split1', 'split2', 'split3', 'split4', 'split5', 'split6'
    var: 'feature_name', 'gene_ensembl', 'gene_symbol', 'identity_embedding_index', 'identity_valid_mask'
    uns: 'gene_mapping:ensembl_to_symbol'
> Using processed cell representations: /scratch/heimdall/shared/cache/new_sctab_full/processed_data/a58817cb-bd44-e997-5a6c-d494b3115f02/data.pkl
> Data is dense and backed, skipping conversion to CSR to keep memory mapping.
> Found already preprocessed anndata: /s

/work/magroup/shahula/single_cell_foundation_models/Heimdall/Heimdall/datasets.py:400: UserWarning: Pre-defined split unavailable, using random 8/1/1 split
  def _setup_random_splits(self):


> Finished Processing Anndata Object:
AnnData object with n_obs × n_vars = 10000 × 19331 backed at '/scratch/heimdall/shared/cache/new_sctab_full/preprocessed_anndata/dd6fa465-72ca-143d-7ae4-8964c61d1ff2/data.h5ad'
    obs: 'soma_joinid', 'is_primary_data', 'dataset_id', 'donor_id', 'assay', 'cell_type', 'development_stage', 'disease', 'tissue', 'tissue_general', 'split', 'split1', 'split2', 'split3', 'split4', 'split5', 'split6'
    var: 'feature_name', 'gene_ensembl', 'gene_symbol'
    uns: 'gene_mapping:ensembl_to_symbol'
> Using processed cell representations: /scratch/heimdall/shared/cache/new_sctab_full/processed_data/a58817cb-bd44-e997-5a6c-d494b3115f02/data.pkl
> Data is dense and backed, skipping conversion to CSR to keep memory mapping.
> Found already preprocessed anndata: /scratch/heimdall/shared/cache/new_sctab_full/preprocessed_anndata/6a4725f5-f683-4a88-21f5-c1b54981659b/data.h5ad


/work/magroup/shahula/single_cell_foundation_models/Heimdall/Heimdall/datasets.py:400: UserWarning: Pre-defined split unavailable, using random 8/1/1 split
  def _setup_random_splits(self):


> Finished Processing Anndata Object:
AnnData object with n_obs × n_vars = 10000 × 19331 backed at '/scratch/heimdall/shared/cache/new_sctab_full/preprocessed_anndata/6a4725f5-f683-4a88-21f5-c1b54981659b/data.h5ad'
    obs: 'soma_joinid', 'is_primary_data', 'dataset_id', 'donor_id', 'assay', 'cell_type', 'development_stage', 'disease', 'tissue', 'tissue_general', 'split', 'split1', 'split2', 'split3', 'split4', 'split5', 'split6'
    var: 'feature_name', 'gene_ensembl', 'gene_symbol', 'identity_embedding_index', 'identity_valid_mask'
    uns: 'gene_mapping:ensembl_to_symbol'
> Using processed cell representations: /scratch/heimdall/shared/cache/new_sctab_full/processed_data/a58817cb-bd44-e997-5a6c-d494b3115f02/data.pkl
> Data is dense and backed, skipping conversion to CSR to keep memory mapping.
> Dataset splits sizes (random):
  train: 8,000
  val: 1,000
  test: 1,000
> Finished setting up datasets (and loaders):
	{'full': PartitionedDataset(size=10,000) wrapping: <Heimdall.cell_repr

/work/magroup/shahula/bin/miniconda3/envs/heimdall_distribution/lib/python3.10/site-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [380]:
loader = cr.dataloaders['train']

In [381]:
from time import time
times = []

start = time()
for batch in loader:
    times.append(time() - start)
    start = time()

print(f'Mean time to fetch batch is {np.mean(times) * 1e3:.4f} ms')

> Found already preprocessed anndata: /scratch/heimdall/shared/cache/new_sctab_full/preprocessed_anndata/dd6fa465-72ca-143d-7ae4-8964c61d1ff2/data.h5ad
> Finished Processing Anndata Object:
AnnData object with n_obs × n_vars = 10000 × 19331 backed at '/scratch/heimdall/shared/cache/new_sctab_full/preprocessed_anndata/dd6fa465-72ca-143d-7ae4-8964c61d1ff2/data.h5ad'
    obs: 'soma_joinid', 'is_primary_data', 'dataset_id', 'donor_id', 'assay', 'cell_type', 'development_stage', 'disease', 'tissue', 'tissue_general', 'split', 'split1', 'split2', 'split3', 'split4', 'split5', 'split6'
    var: 'feature_name', 'gene_ensembl', 'gene_symbol'
    uns: 'gene_mapping:ensembl_to_symbol'
> Using processed cell representations: /scratch/heimdall/shared/cache/new_sctab_full/processed_data/a58817cb-bd44-e997-5a6c-d494b3115f02/data.pkl
> Data is dense and backed, skipping conversion to CSR to keep memory mapping.
> Finished setting up datasets (and loaders):
	{'full': PartitionedDataset(size=10,000) wra

/work/magroup/shahula/bin/miniconda3/envs/heimdall_distribution/lib/python3.10/site-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


> Found already preprocessed anndata: /scratch/heimdall/shared/cache/new_sctab_full/preprocessed_anndata/6a4725f5-f683-4a88-21f5-c1b54981659b/data.h5ad
> Finished Processing Anndata Object:
AnnData object with n_obs × n_vars = 10000 × 19331 backed at '/scratch/heimdall/shared/cache/new_sctab_full/preprocessed_anndata/6a4725f5-f683-4a88-21f5-c1b54981659b/data.h5ad'
    obs: 'soma_joinid', 'is_primary_data', 'dataset_id', 'donor_id', 'assay', 'cell_type', 'development_stage', 'disease', 'tissue', 'tissue_general', 'split', 'split1', 'split2', 'split3', 'split4', 'split5', 'split6'
    var: 'feature_name', 'gene_ensembl', 'gene_symbol', 'identity_embedding_index', 'identity_valid_mask'
    uns: 'gene_mapping:ensembl_to_symbol'
> Using processed cell representations: /scratch/heimdall/shared/cache/new_sctab_full/processed_data/a58817cb-bd44-e997-5a6c-d494b3115f02/data.pkl
> Data is dense and backed, skipping conversion to CSR to keep memory mapping.
> Finished setting up datasets (and load

/work/magroup/shahula/bin/miniconda3/envs/heimdall_distribution/lib/python3.10/site-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Mean time to fetch batch is 65.0973 ms
